In [6]:
import pandas as pd


In [23]:
orders_df = pd.read_excel('Заявка от клиента.xlsx')
suppliers_df = pd.read_excel('Предложение от поставщиков.xlsx')
delivery_df = pd.read_excel('Доставка из других городов.xlsx')

In [ ]:
suppliers_df.fillna(method='ffill', inplace=True)

In [18]:
delivery_df

,Поставщик,Доставка
0,Все инструменты,500
1,Сатурн,200
2,Петрович,1000
3,Гарвин,500


In [9]:
orders_df

,№,Номенклатура,Количество
0,1,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3
1,2,"Ключ трубный CrV 90 градусов 2,0"" 12\n\n\n\n\n",1
2,3,"КТР-90 ключ трубный, прямые губки",1
3,4,Ключ разводной КР-30 окс\n\n\n\n\n,2
4,5,Фонарь прожект./кемп.,3
5,6,Ключ разводной тонкие губки 50мм,1


In [26]:
suppliers_df

,№,Номенклатура,Количество (заявка),Поставщик,Город,Цена поставщика,Количество (поставщик),"Сроки поставки, дни"
0,1.0,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3.0,Все инструменты,Спб,499.00,3,1.0
1,1.0,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3.0,Сатурн,Спб,603.84,3,1.0
2,1.0,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3.0,Петрович,Спб,471.00,3,1.0
3,1.0,Ключ трубный рычажный 1 S-обр\n\n\n\n\n\n,3.0,Гарвин,Спб,3504.00,3,3.0
4,2.0,"Ключ трубный CrV 90 градусов 2,0"" 12\n\n\n\n\n",1.0,Сатурн,Спб,1403.58,1,1.0
5,2.0,"Ключ трубный CrV 90 градусов 2,0"" 12\n\n\n\n\n",1.0,Гарвин,Спб,3968.00,1,3.0
6,2.0,"Ключ трубный CrV 90 градусов 2,0"" 12\n\n\n\n\n",1.0,Петрович,Спб,1408.00,1,3.0
7,3.0,"КТР-90 ключ трубный, прямые губки",1.0,Все инструменты,Спб,2480.00,1,1.0
8,3.0,"КТР-90 ключ трубный, прямые губки",1.0,Гарвин,Спб,10048.00,1,3.0
9,3.0,"КТР-90 ключ трубный, прямые губки",1.0,Сатурн,Спб,2750.00,1,3.0


In [27]:
# Список уникальных товаров из заказа
order_items = orders_df['Номенклатура'].unique()

# Группируем данные о поставщиках по имени поставщика
grouped_suppliers = suppliers_df.groupby('Поставщик')

# Словарь для хранения стоимости заказа для каждого поставщика
supplier_costs = {}

# Проходим по каждому поставщику и проверяем наличие всех товаров
for supplier, group in grouped_suppliers:
    supplier_items = group['Номенклатура'].unique()
    
    # Проверяем, есть ли у поставщика все товары из заказа
    if all(item in supplier_items for item in order_items):
        total_cost = 0
        for _, order_row in orders_df.iterrows():
            item = order_row['Номенклатура']
            quantity = order_row['Количество']
            # Получаем цену товара у текущего поставщика и проверяем наличие достаточного количества
            available_items = group[(group['Номенклатура'] == item) & (group['Количество (поставщик)'] >= quantity)]
            if not available_items.empty:
                price = available_items['Цена поставщика'].values[0]
                total_cost += price * quantity
            else:
                total_cost = float('inf')
                break
        
        # Добавляем стоимость доставки
        delivery_cost = delivery_df[delivery_df['Поставщик'] == supplier]['Доставка'].values[0]
        total_cost += delivery_cost
        
        if total_cost < float('inf'):
            supplier_costs[supplier] = total_cost

# Находим поставщика с минимальной стоимостью заказа
if supplier_costs:
    best_supplier = min(supplier_costs, key=supplier_costs.get)
    best_cost = supplier_costs[best_supplier]
    
    print(f"Лучший поставщик: {best_supplier} с общей стоимостью заказа: {best_cost}")
else:
    print("Нет ни одного поставщика, у которого есть все товары из заказа")

Лучший поставщик: Сатурн с общей стоимостью заказа: 10311.46


In [21]:
import pandas as pd
import itertools


# Список товаров и их количество из заказа
order_items = orders_df.to_dict('records')

# Группировка данных о поставщиках по названию товара
grouped_suppliers = suppliers_df.groupby('Номенклатура')

# Словарь для хранения доступных поставщиков и их товарного запаса для каждого товара
suppliers_dict = {}
for item in order_items:
    item_name = item['Номенклатура']
    if item_name in grouped_suppliers.groups:
        suppliers_info = grouped_suppliers.get_group(item_name)[['Поставщик', 'Цена поставщика', 'Количество (поставщик)']]
        suppliers_info = suppliers_info[suppliers_info['Количество (поставщик)'] >= item['Количество']]
        if not suppliers_info.empty:
            suppliers_dict[item_name] = suppliers_info.to_dict('records')

# Функция для расчета стоимости комбинации
def calculate_cost(combination):
    total_cost = 0
    unique_suppliers = set()
    for item, supplier_info in zip(order_items, combination):
        quantity = item['Количество']
        total_cost += supplier_info['Цена поставщика'] * quantity
        unique_suppliers.add(supplier_info['Поставщик'])
    
    # Учитываем стоимость доставки от каждого уникального поставщика
    delivery_cost = sum(delivery_df[delivery_df['Поставщик'] == supplier]['Доставка'].values[0] for supplier in unique_suppliers)
    total_cost += delivery_cost
    
    return total_cost, unique_suppliers

# Поиск всех возможных комбинаций
all_combinations = itertools.product(*suppliers_dict.values())

# Переменные для хранения наилучшей комбинации
best_combination = None
lowest_cost = float('inf')
best_suppliers = None

# Проходим по всем комбинациям и находим наилучшую
for combination in all_combinations:
    cost, suppliers = calculate_cost(combination)
    if cost < lowest_cost:
        lowest_cost = cost
        best_combination = combination
        best_suppliers = suppliers

# Вывод результата
if best_combination:
    print(f"Наилучшая комбинация поставщиков с общей стоимостью: {lowest_cost}")
    for item, supplier_info in zip(order_items, best_combination):
        print(f"Товар: {item['Номенклатура']}, Поставщик: {supplier_info['Поставщик']}, Цена: {supplier_info['Цена поставщика']}, Количество: {item['Количество']}")
    print(f"Уникальные поставщики: {best_suppliers}")
else:
    print("Нет доступных комбинаций для выполнения заказа.")


Наилучшая комбинация поставщиков с общей стоимостью: 10063.939999999999
Товар: Ключ трубный рычажный 1 S-обр





, Поставщик: Петрович, Цена: 471.0, Количество: 3
Товар: Ключ трубный CrV 90 градусов 2,0" 12




, Поставщик: Сатурн, Цена: 1403.58, Количество: 1
Товар: КТР-90 ключ трубный, прямые губки, Поставщик: Все инструменты, Цена: 2480.0, Количество: 1
Товар: Ключ разводной КР-30 окс




, Поставщик: Сатурн, Цена: 175.68, Количество: 2
Товар:  Фонарь прожект./кемп. , Поставщик: Петрович, Цена: 511.0, Количество: 3
Товар: Ключ разводной тонкие губки 50мм, Поставщик: Все инструменты, Цена: 1183.0, Количество: 1
Уникальные поставщики: {'Сатурн', 'Все инструменты', 'Петрович'}
